<a href="https://colab.research.google.com/github/egiperez49/AgricultureNet/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 80.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
#
import rasterio
from pathlib import Path

In [ ]:
# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# prompt: descomprimir un archivo .rar en la carpeta content

!pip install unrar
!unrar x /content/recortes_mascara.rar /content/msk/



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/recortes_mascara.rar

Extracting  /content/msk/img-0_3.tif                                       0%  OK 
Extracting  /content/msk/img-0_4.tif                                       0%  OK 
Extracting  /content/msk/img-0_5.tif                                       0%  OK 
Extracting  /content/msk/img-0_6.tif                                       0%  OK 
Extracting  /content/msk/img-0_7.tif                                       0%  OK 
Extracting  /content/msk/img-0_8.tif                                       0%  OK 
Extracting  /content/msk/img-0_9.tif                                       0%  OK 
Extracting  /content/msk/img-0_10.tif                                      0%  OK 
Extracting  /content/msk/img-0_11.tif                                      1%  OK 
Extracting  /content/msk/img-0_12.tif            

In [ ]:
# Dataset and DataLoader
PATH = Path("/content/drive/","IM10")  # Ruta donde estan todas las carpetas
TRAIN_PATH = Path("/content/drive/","IM10", "imgs")
TRAIN_MASK_PATH = Path("/content/drive/","IM10", "msks")
TEST_PATH = Path("/content/drive/My Drive/","IM10/test")

In [ ]:
# Dataset and DataLoader New //Modificado Eperez
PATH_n = Path("/content/")  # Ruta donde estan todas las carpetas
TRAIN_PATH_n = Path("/content/", "imgs")
TRAIN_MASK_PATH_n = Path("/content/", "msk")
TEST_PATH_n = Path("/content/","test")

In [ ]:
# Dataset
# Creando nuestro Dataset


class Satelital_Dataset(Dataset):

    def __init__(self, data, masks=None, img_transforms=None, mask_transforms=None):
        super().__init__()
        '''
        data: train data path
        mask: train masks path
        '''
        self.train_data = data
        self.train_masks = masks

        self.img_transforms = img_transforms
        self.mask_transforms = mask_transforms

        self.images = sorted(os.listdir(self.train_data))
        self.masks = sorted(os.listdir(self.train_masks))

    def __len__(self):
        if self.train_masks is not None:
            assert len(self.images) == len(
                self.masks), "ERROR: No hay el mismo nro de imagenes y mascaras"
        return len(self.images)

    def __getitem__(self, index):
        '''
        index: es el indice de la imagen que queremos obtener
        '''
        image_name = os.path.join(self.train_data, self.images[index])
        # Leyendo la imagen con rasterio
        with rasterio.open(image_name) as dataset:
            img = dataset.read()
        # Mover el eje de las bandas al final
        img = np.moveaxis(img, 0, -1)  # img.shape: (256, 256, N)
        trans = T.ToTensor()
        if self.img_transforms is not None:
            img = self.img_transforms(img)
        else:
            img = trans(img)
        # Normalizando la imagen (antes no estaba normalizada y la mascara si, pero necesito mi data normalizada)
        img_max = img.max().item()
        img /= img_max
        # print("Shape img: ", img.shape, type(img))
        if self.train_masks is not None:
            mask_name = os.path.join(self.train_masks, self.masks[index])
            # Leyendo la mascara
            with rasterio.open(mask_name) as dataset_2:
                mask = dataset_2.read()
            # Mover el eje de las bandas al final
            mask = np.moveaxis(mask, 0, -1)  # img.shape: (256, 256, N)
            if self.mask_transforms is not None:
                mask = self.mask_transforms(mask)
            else:
                mask = trans(mask)
            # Normalizando la mascara (esto lo elimine ya que se supone que la mascara es una mascara binaria)
            """mask_max = mask.max().item()
            mask /= mask_max"""
            return img, mask

        return img

In [ ]:
# Transformaciones
transform_data = T.Compose([T.ToTensor()])

In [ ]:
# DataLoaders
#from satelital_data_set import Satelital_Dataset
full_dataset = Satelital_Dataset(TRAIN_PATH_n,
                                 TRAIN_MASK_PATH_n,
                                 img_transforms=transform_data,
                                 mask_transforms=transform_data)

In [ ]:
BATCH_SIZE = 4
TRAIN_SIZE = int(len(full_dataset)*0.8)
VAL_SIZE = len(full_dataset) - TRAIN_SIZE

In [ ]:
train_dataset, val_dataset = random_split(full_dataset, [TRAIN_SIZE, VAL_SIZE])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
for i in range(len(full_dataset)):
    img, mask = full_dataset[i]
    print(f"Valores de la máscara {i}: \n {mask}")

Valores de la máscara 0: 
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
Valores de la máscara 1: 
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
Valores de la máscara 2: 
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
Valores de la máscara 3: 
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  .

In [ ]:
imgs, masks = next(iter(train_loader))
print(imgs.shape, masks.shape)

torch.Size([4, 4, 256, 256]) torch.Size([4, 1, 256, 256])


In [ ]:
class batchnorm_relu(nn.Module):
    def __init__(self, in_c):
        super().__init__()

        self.bn = nn.BatchNorm2d(in_c)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.bn(inputs)
        x = self.relu(x)
        return x

class residual_block(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super().__init__()

        """ Convolutional layer """
        self.b1 = batchnorm_relu(in_c)
        self.c1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, stride=stride)
        self.b2 = batchnorm_relu(out_c)
        self.c2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, stride=1)

        """ Shortcut Connection (Identity Mapping) """
        self.s = nn.Conv2d(in_c, out_c, kernel_size=1, padding=0, stride=stride)

    def forward(self, inputs):
        x = self.b1(inputs)
        x = self.c1(x)
        x = self.b2(x)
        x = self.c2(x)
        s = self.s(inputs)

        skip = x + s
        return skip

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.r = residual_block(in_c+out_c, out_c)

    def forward(self, inputs, skip):
        x = self.upsample(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.r(x)
        return x

class build_resunet(nn.Module):
    def __init__(self, in_channels, num_classes=1):
        super().__init__()

        """ Encoder 1 """
        self.c11 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.br1 = batchnorm_relu(64)
        self.c12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.c13 = nn.Conv2d(in_channels, 64, kernel_size=1, padding=0)

        """ Encoder 2 and 3 """
        self.r2 = residual_block(64, 128, stride=2)
        self.r3 = residual_block(128, 256, stride=2)

        """ Bridge """
        self.r4 = residual_block(256, 512, stride=2)

        """ Decoder """
        self.d1 = decoder_block(512, 256)
        self.d2 = decoder_block(256, 128)
        self.d3 = decoder_block(128, 64)

        """ Output """
        self.output = nn.Conv2d(64, num_classes, kernel_size=1, padding=0)
        if num_classes == 1:
          self.activation = nn.Sigmoid()
        elif num_classes > 1:
          self.activation = nn.Softmax(dim=1)

    def forward(self, inputs):
        """ Encoder 1 """
        x = self.c11(inputs)
        x = self.br1(x)
        x = self.c12(x)
        s = self.c13(inputs)
        skip1 = x + s

        """ Encoder 2 and 3 """
        skip2 = self.r2(skip1)
        skip3 = self.r3(skip2)

        """ Bridge """
        b = self.r4(skip3)

        """ Decoder """
        d1 = self.d1(b, skip3)
        d2 = self.d2(d1, skip2)
        d3 = self.d3(d2, skip1)

        """ output """
        output = self.output(d3)
        if hasattr(self, 'activation'):
            output = self.activation(output)
        return output

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.auto import tqdm  # Importa tqdm desde tqdm.auto

In [ ]:
IN_CHANNELS = 4
NUM_CLASSES = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 3

#1. Instanciar el modelo
model = build_resunet(in_channels=IN_CHANNELS, num_classes=NUM_CLASSES)

# Convertir el modelo a float16
model = model.type(torch.float16)

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Definir la función de pérdida.
if NUM_CLASSES == 1:
    criterion = nn.BCEWithLogitsLoss() # Para segmentación binaria
else:
    criterion = nn.CrossEntropyLoss()

# 3. Definir el optimizador
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 4. Ciclo de entrenamiento
for epoch in range(NUM_EPOCHS):
    model.train()  # Poner el modelo en modo de entrenamiento
    train_loss = 0.0
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")):
        # Convertir inputs y targets a float16 y moverlos a la GPU
        inputs = inputs.type(torch.float16).to(device)
        targets = targets.type(torch.float16).to(device)


        # Imprimir máximos y mínimos de imgs (inputs)
       # print("Máximo valor en imgs:", inputs.max().item())
       # print("Mínimo valor en imgs:", inputs.min().item())

        # Imprimir máximos y mínimos de masks (targets)
      #  print("Máximo valor en masks:", targets.max().item())
       # print("Mínimo valor en masks:", targets.min().item())

      #  print("Valores únicos en la máscara:", torch.unique(targets))

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache() # Liberar memoeria caché de cuda

        train_loss += loss.item() * inputs.size(0)

    # 5. Validación
    model.eval()  # Poner el modelo en modo de evaluación
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc="Validating"):
            # Convertir inputs y targets a float16 y moverlos a la GPU
            inputs = inputs.type(torch.float16).to(device) # <-- Agregar esta línea
            targets = targets.type(torch.float16).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)

    # Calcular pérdidas promedio
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # 6. Guardar el modelo (opcional)
    torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")

print("Entrenamiento finalizado!")

Epoch 1/3:   0%|          | 0/125 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1/3, Train Loss: nan, Val Loss: nan


Epoch 2/3:   0%|          | 0/125 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 2/3, Train Loss: nan, Val Loss: nan


Epoch 3/3:   0%|          | 0/125 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 3/3, Train Loss: nan, Val Loss: nan
Entrenamiento finalizado!
